In [4]:
import xarray as xr
import numpy as np

import pyxpcm
from pyxpcm.models import pcm

# ISAS dataset
file_path = '/home5/pharos/REFERENCE_DATA/OCEAN_REP/ISAS/ISAS15/ANA/ISAS_DT/ARGO_ONLY/2015/'

#time range
start_date = 20150115
end_date = 20150115
time_interval = 100 # TODO: easier way to get time interval, date type

#spatial extent
lon_extent = [-6,35]
lat_extent = [30,45]

# get list of paths
dates_range = np.arange(start_date, end_date + time_interval, time_interval)

ds_paths = []
for dt in dates_range:
    ds_paths.append([file_path + 'ISAS15_ARGO_' + str(dt) +'_fld_TEMP.nc'])

#open all data files
ds = xr.open_mfdataset(ds_paths,combine='by_coords', concat_dim='time')

ds = ds.sel(latitude=slice(lat_extent[0],lat_extent[1]), longitude=slice(lon_extent[0], lon_extent[1])) 
# !!!!!! after changing depth values, attributes disappear (ds['depth'].values = -np.abs(ds['depth'].values) is not working)
ds['depth'] = -np.abs(ds['depth'].values)

#Choise of z and pcm features (very important to make a good choise) 
z = np.arange(0.,-2000,-10.)
pcm_features = {'temperature': z}
K=6
m = pcm(K=K, features=pcm_features)

# fit
features_in_ds = {'temperature': 'TEMP'}
features_zdim='depth'
m.fit(ds, features=features_in_ds, dim=features_zdim)

#classify data
m.predict(ds, features=features_in_ds, dim=features_zdim, inplace=True);

# !!!!!! after predict no attributes in coordinates
ds.longitude.attrs

{}

In [9]:
import xarray as xr
import numpy as np

import argopy
from argopy import DataFetcher as ArgoDataFetcher
argo_loader = ArgoDataFetcher()

# get data
ds = argo_loader.region([-6, 35, 30, 46, 0, 1000, '2012', '2013']).to_xarray()

dsp = ds.argo.point2profile()
# !!!!!! after point2profile no attributes in coordinates
dsp.LONGITUDE.attrs

{}